In [ ]:
import pandas as pd
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.color import rgb2gray
import itertools
import os
from sklearn.model_selection import train_test_split
import time
from sklearn import preprocessing
from sklearn import metrics

In [ ]:
#test-train split
imageData = os.listdir("./images")
imageData = [imd for imd in imageData if imd.split('.')[-1] == 'jpg']
imageLabel = os.listdir("./labels")
imageLabel = [iml for iml in imageLabel if iml.split('.')[-1] == 'jpg']
#ensure image and labels correspond
imageData.sort()
imageLabel.sort()
trainImage,validImage,trainLable,validLabel = train_test_split(imageData,imageLabel,test_size=0.2,shuffle=True,random_state=2)
def addPrefix(x):
    pre = x.split("-")[1][0:3]
    if(pre == "vol"):
        return "./images/" + x
    elif(pre == "lab"):
        return "./labels/" + x
    else:
        return "Invalid input"
tmp = [trainImage,validImage,trainLable,validLabel]

trainImage = list(map(addPrefix,trainImage))
validImage = list(map(addPrefix,validImage))
trainLable = list(map(addPrefix,trainLable))
validLabel = list(map(addPrefix,validLabel))




In [ ]:
def histogram(greyLevel, img):
    
    img = img.reshape(img.shape[0]*img.shape[1])
    num = 0
    totalPixel = 9
    for i in range (totalPixel):
        if(img[i] == greyLevel):
            num = num+1
            
    return num/totalPixel
    

In [ ]:
pick = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]

In [ ]:
def mean(img):
    mean_val = 0
    for i in range (256):
        mean_val = mean_val + i*histogram(i, img)
        
    return mean_val
        

In [ ]:
def variance(img):
    variance_val = 0
    mean_val = mean(img)
    for i in range (256):
        variance_val = ((i-mean_val) ** 2)*histogram(i, img) + variance_val
        
    return variance_val

In [ ]:
def skewness(img):
    skewness_val = 0
    mean_val = mean(img)
    variance_val = variance(img)
    variance_val = variance_val ** (3/2)
    for i in range (256):
        skewness_val = ((i-mean_val) ** 3)*histogram(i, img) + skewness_val
        
    skewness_val = skewness_val / variance_val
    return skewness_val
    

In [ ]:
def peak(img):
    mean_val = mean(img)
    variance_val = variance(img) ** 2
    peak_val = 0
    for i in range (256):
        peak_val = peak_val + (((i - mean_val) ** 4)*histogram(i, img) - 3)
        
    peak_val = peak_val/variance_val
    return peak_val
    

In [ ]:
def gray_level_vector(img, center_x, center_y):
   
    #patch_size = 5
    #gray= cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    T = np.zeros((1,17))
    
    x1 = center_x[0] - 1
    y1 = center_y[0] - 1
    x2 = center_x[0] + 2
    y2 = center_y[0] + 2
    
    patch = img[x1:x2, y1:y2]

    
    for j in range(len(pick)):
            T[0][j] = patch[pick[j][0],pick[j][1]]
   
    return T, patch
        

In [ ]:
def fact(mean,h,n):

    return fact_iter(0,0,mean,h,n)

def fact_iter(product, count, mean,h,n):

    if count >= len(n):

        return product

    return fact_iter(product+(n[count]-mean)**h, count+1,mean,h,n)

In [ ]:

def T_gen(img, center_x, center_y):
    
    T, patch = gray_level_vector(img, center_x, center_y)
    
    T[0][9] = int(mean(patch))    
    T[0][10] = round(variance(patch),3)    
    T[0][11] = round(skewness(patch),3)   
    T[0][12] = int(peak(patch))
    T[0][13] = fact(0,2,T[0][:9])
    T[0][14] = fact(T[0][9],2,T[0][:9])/9
    T[0][15] = fact(T[0][9],3,T[0][:9])/9
    T[0][16] = fact(T[0][9],4,T[0][:9])/9
    
    
    return T

In [ ]:

def patchList(img,label):
    print(".........")
    h,w = img.shape
    patchImage = []
    patchLabel = []
    for i in range(3000):
        
        randomHeight = np.random.randint(low = 1,high = h-1,size = 1)        
        randomWeight = np.random.randint(low = 1,high = w-1,size = 1)
        
        labelPixel = label[randomWeight,randomHeight]
        patchLabel.append(labelPixel)
        
        T = T_gen(img,randomWeight,randomHeight)
        patchImage.append(T)
        
    return np.array(patchImage), np.array(patchLabel)
                

In [ ]:

def final_generator(list1,list2):
    for_train = []
    for_test = []
    for path in range(len(list1)):
        img = cv2.imread(list1[path],1)
        img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        ret1,lab1 = cv2.threshold(cv2.imread(list2[path],0),127,255,cv2.THRESH_BINARY)
        Train,label_patch = patchList(img,lab1)
       
        aft = Train.reshape((Train.shape[0],-1))
        aft = preprocessing.maxabs_scale(aft,axis=0, copy=True)
        for_train.append(aft)
        for_test.append(label_patch)
        #print("for_train",np.array(for_train).shape)
        affor_train = np.array(for_train).reshape((-1,17))
        affor_test = np.array(for_test).reshape((-1,1))
    return affor_train,affor_test

In [ ]:

TTrain,Tlabel = final_generator(trainImage,trainLable)

In [ ]:
def train_model(X, y):
    start = time.time()
    from sklearn.svm import SVC
    print ('[INFO] Training Support Vector Machine model.')
    model = SVC(gamma = "auto",kernel = 'rbf')
    model.fit(X, y)

    print ('[INFO] Model training complete.')
#     print ('[INFO] Training Accuracy: %.2f' %model.score(X, y))
    stop = time.time()
    d = stop - start
    print('Running duration is {}'.format(d))
    return model

In [ ]:
model = train_model(TTrain,Tlabel)

In [ ]:
def test_model(X, y, model):
    
    pred = model.predict(X)
    precision = metrics.precision_score(y, pred, average='weighted', labels=np.unique(pred))
    recall = metrics.recall_score(y, pred, average='weighted', labels=np.unique(pred))
    f1 = metrics.f1_score(y, pred, average='weighted', labels=np.unique(pred))
    accuracy = metrics.accuracy_score(y, pred)
    
    print ('--------------------------------')
    print ('[RESULTS] Accuracy: %.2f' %accuracy)
    print ('[RESULTS] Precision: %.2f' %precision)
    print ('[RESULTS] Recall: %.2f' %recall)
    print ('[RESULTS] F1: %.2f' %f1)
    print ('--------------------------------')

In [ ]:
def pre(X,model):
    pred = model.predict(X)
    return pred

In [ ]:
TTest,TTlabel = final_generator(validImage[2:3],validLabel[2:3])

In [ ]:
test_model(TTest,TTlabel,model)

In [ ]:
import time
def evalPatch(testImg,patch_size):
    h,w = testImg.shape
    #patch_size should be odd
    if(patch_size%2 == 0):
        patch_size += 1
    #image side cut
    cut = patch_size // 2  
    patches = []
    print('cut is %d and limit is %d'%(cut,w-cut))
    for row in range(1,511):
        #start = time.time()
        print("........")
        row = [row]
        for col in range(1,511):
            col = [col]
            T = T_gen(testImg,row,col)
            #T = T_gen(testImg,row,col)
            patches.append(T)
        #stop = time.time()
        #dur = stop-start
        print(row)
    return np.array(patches)

In [ ]:
img = cv2.imread(trainImage[2:3][0],0)
ee = evalPatch(img,3)


In [ ]:
aft = ee.reshape((ee.shape[0],-1))
print(aft.shape)
aft = preprocessing.maxabs_scale(aft,axis=0, copy=True)

In [ ]:
print(aft.shape)

In [ ]:
pret =  pre(aft,model)

In [ ]:
pp = pret.reshape((510,510))

In [ ]:
img_blur_G = cv2.GaussianBlur(pp,(5,5),0)
imm = pp/255

In [ ]:
plt.figure()
plt.imshow(pp,'gray')
plt.show()

In [ ]:
plt.figure()
plt.imshow(img_blur_G,'gray')
plt.show()

In [ ]:
ret,thh = cv2.threshold(img_blur_G,160,255,cv2.THRESH_BINARY)

In [ ]:
import tifffile
img = (img_blur_G/255).astype('float32')
tiff_list = img
tifffile.imsave('./%d_predict.tif'%3333333, tiff_list)